In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
vv = [[0, 1, 2, 3, 4]]

In [210]:
from boundary_matrix import build_boundary_matrix
from boundary_matrix import lower_dimensions_index
from boundary_matrix import replace_list
from boundary_matrix import vector_vector_T
import numpy as np
import os
import pandas as pd
#import phat

In [211]:
for i, b in enumerate(build_boundary_matrix(vv)):
    print(i, b[0], b[1])

[()]
[()]
[(0,)]
[0]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(1,)]
[1]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(0,)]
[0]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(2,)]
[2]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(0,)]
[0]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(3,)]
[3]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(0,)]
[0]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(4,)]
[4]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(1,)]
[1]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(2,)]
[2]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(1,)]
[1]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(3,)]
[3]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(1,)]
[1]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(4,)]
[4]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(2,)]
[2]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(3,)]
[3]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(2,)]
[2]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(4,)]
[4]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(3,)]
[3]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(4,)]
[4]
[]
[]
[]
[]
[]
[]
[()]
[()]
[(0,), (1,)]
[0, 1]
[(0, 1)]
[5]
[]
[]
[]
[]
[()]
[()]
[(0,), (2,)]
[0, 2]
[(0, 2)]
[6]
[]
[]
[]
[]
[()]
[()]
[(1,), (2,)]
[1, 2]
[(1, 2)]
[9]
[]
[]
[]
[]
[()]
[()]
[(0,), (1,)]
[0, 1]
[(0, 1)]
[5]
[]
[]
[]
[]


In [ ]:
data = os.path.join(os.pardir, os.pardir, "train_test_data", "unsplit_no_totals.csv")

In [ ]:
df = pd.read_csv(data)

In [ ]:
dxes = {("Anx", "adhd", "asd"):
           {(0,0,0),
           (1,0,0),
           (0,1,0),
           (0,0,1),
           (1,1,0),
           (1,0,1),
           (0,1,1),
           (1,1,1)}
       }

In [ ]:
subdfs = {}
for k in dxes:
    for r in dxes[k]:
        d = set(zip(k, r))
        subdfs[str(dict(d))] = df.loc[
            eval(
                " & ".join(
                    {
                        "(df[\"{0}\"] == {1})".format(key, value) for key, value in d
                    }
                )
            )
        ].copy()

In [ ]:
for n in subdfs:
    print(n, ":", len(subdfs[n]))

In [ ]:
bmdir = os.path.join(os.pardir, os.pardir, "boundary_matrices")
if not os.path.exists(bmdir):
    os.makedirs(bmdir)

In [ ]:
matrices = []
for n in subdfs:
    df_in_use = subdfs[n].loc[subdfs[n]["train"]].drop(
        ["Dx", "Anx", "adhd", "asd", "train", "Sex", "Age", "EID"],
        axis=1
    ).copy()
    df_in_use.reset_index(inplace=True, drop=True)
    for col in [col for col in df_in_use.columns if col != "EID"]:
        df_in_use[col] = (df_in_use[col] == max(df_in_use[col]))
    bmatrix = build_boundary_matrix(vector_vector_T(df_in_use))
    with open(os.path.join(
        bmdir,
        "{0}.txt".format(replace_list(n.replace(" ", "_"),[":", ",", "'", "{", "}"], ""))
    ), "w") as of:
        of.write(str(bmatrix))
    matrices.append(bmatrix)

In [ ]:
for n in subdfs:
    df_in_use = subdfs[n].loc[subdfs[n]["train"]].drop(
        ["Dx", "Anx", "adhd", "asd", "train", "Sex", "Age", "EID"],
        axis=1
    ).copy()
    df_in_use.reset_index(inplace=True, drop=True)
    for col in [col for col in df_in_use.columns if col != "EID"]:
        df_in_use[col] = (df_in_use[col] == max(df_in_use[col]))
    boundary_matrix = phat.boundary_matrix(representation = phat.representations.vector_vector)

    boundary_matrix.columns = build_boundary_matrix(vector_vector_T(df_in_use))
    
    print("The boundary matrix has %d columns:" % len(boundary_matrix.columns))
    for col in boundary_matrix.columns:
        s = "Column %d represents a cell of dimension %d." % (col.index, col.dimension)
        if (col.boundary):
            s = s + " Its boundary consists of the cells " + " ".join([str(c) for c in col.boundary])
        print(s)
    print("Overall, the boundary matrix has %d entries." % len(boundary_matrix))

    pairs = boundary_matrix.compute_persistence_pairs()

    pairs.sort()

    print()
    print("There are %d persistence pairs: " % len(pairs))
    for pair in pairs:
        print("Birth: %d, Death: %d" % pair)